In [1]:
import re

import numpy as np

from matplotlib import pyplot as plt
from IPython.display import display, Markdown
from pathlib import Path
from collections import defaultdict

In [2]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400

def show():
    plt.savefig(f"{show.fig_counter}.pdf", dpi='figure', transparent=True, bbox_inches='tight')
    show.fig_counter += 1
    show.plt_show()

show.fig_counter = 0
show.plt_show = plt.show

plt.show = show

In [3]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

In [4]:
BIODIVERSITY_PATTERN = re.compile(r"The simulation resulted in a biodiversity of (\d+) unique species\.")
EXECUTION_PATTERN = re.compile(r"The simulation took:\n - initialisation: ([^\n]+)\n - execution: ([^\n]+)\n - cleanup: ([^\n]+)\n")

In [5]:
TIME_PATTERN = re.compile(r"(\d+(?:\.\d+)?)([^\d]+)")
TIME_UNITS = {
    "ns": 0.000000001,
    "µs": 0.000001,
    "ms": 0.001,
    "s": 1.0,
}

def parse_time(time_str):
    match = TIME_PATTERN.match(time_str)
    
    if match is None:
        return None
    
    return float(match.group(1)) * TIME_UNITS[match.group(2)]

In [6]:
DOMAIN_DIR_PATTERN = re.compile(r"([+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?)")
DOMAIN_FILE_PATTERN = re.compile(r"([+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?)/limit\.o\d+\.\d+")

In [7]:
results = defaultdict(list)

for path in Path(".").iterdir():
    match = DOMAIN_DIR_PATTERN.match(str(path))
    
    if match is None:
        continue
    
    print(path)
    
    for path in Path(path).iterdir():
        match = DOMAIN_FILE_PATTERN.match(str(path))
        if match is None:
            continue
            
        sample = float(match.group(1))

        with open(path) as file:
            stdout = file.read()

        match = EXECUTION_PATTERN.search(stdout)
        if match is None:
            print(path)
            continue
        initialisation = parse_time(match.group(1))
        execution = parse_time(match.group(2))
        cleanup = parse_time(match.group(3))
        
        results[sample].append((initialisation, execution, cleanup))

for sample, measures in results.items():
    print(sample, len(measures))

1e10
7.1e10
1e9
10000000000.0 1000
71000000000.0 6390
1000000000.0 100
